In [1]:
import pandas as pd
import numpy as np

In [2]:
def calculate_caim(xj, yj, D):
    sp = [0] + [xj.searchsorted(point) for point in D[1:-1]] + [len(xj)]
    n = len(sp) - 1
    isum = 0
    for j in range(n):
        init = sp[j]
        fin = sp[j + 1]
        segment = yj[init:fin]  
        Mr = segment.shape[0]
        if Mr == 0:
            continue
        _, counts = np.unique(segment, return_counts=True)
        maxr = counts.max()
        isum += (maxr / Mr) * maxr
    return isum / n if n > 0 else 0

In [3]:
def caim(df, verbose=False):
    json = {} #aquí se van a guardar las particiones
    label_column = df.columns[-1]  # Selecciona automáticamente la última columna como etiquetas
    y = df[label_column]
    clases = y.unique()
    min_splits = clases.shape[0]
    if verbose:
        print(f"hay un máximo de {min_splits} cortes")
    
    # Procesa todas las columnas excepto la última
    for column in df.columns[:-1]:
        xj_sorted = df[column].sort_values() #ordena los valores de la columna
        yj_sorted = y.reindex(xj_sorted.index)  # Reordena las etiquetas de acuerdo con el orden de xj_sorted
        
        division_points = xj_sorted.unique()[1:-1] #todos los posibles puntos de corte
        gc = -1 #global caim inicia -1 para asegurar que el primer caim lo reemplace
        D = [xj_sorted.iloc[0], xj_sorted.iloc[-1]] #primer corte: [min,max]
        bc = 0 #best caim, el mejor caim inicia en 0
        mejor_D = D.copy() # el mejor D es el inicial, luego será reemplazado
        k = 1 #valor que servirá de criterio de paro
        
        while k <= min_splits and (gc < bc or len(division_points) > 0):
            #es poco probable que los primeros valores sean útiles para el corte
            #por eso se hace una permutación para hallar el mejor caim antes
            midpoints = np.random.permutation(division_points).tolist() 
            best_caim = 0
            k += 1
            if verbose:
                print(f"ahora k vale {k}")
            while midpoints:
                sp = midpoints.pop() #sacamos el valor para hacer corte
                D_temp = D.copy()
                D_temp.append(sp) #cortamos el principal scheeme
                D_temp.sort()
                caim_value = calculate_caim(xj_sorted, yj_sorted, D_temp)
                if verbose:
                    print(f"{D_temp} tiene caim de: {caim_value}")
                if caim_value > bc: #si el caim es mejor cambiamos bc y D
                    if verbose:
                        print(f"Este caim es mejor, se actualiza best caim a {caim_value}")
                    bc = caim_value
                    mejor_D = D_temp
            if bc > gc:
                gc = bc #si el mejor caim es el mejor reemplaza al global
                D = mejor_D #el nuevo scheeme remplaza al anterior
                if verbose:
                    print(f"se cambia el D, antes era {D}, ahora es {mejor_D}")
        json[column] = D
        print(f"La columna {column} tiene global caim de {gc}\n")
    return json

In [4]:
def discretize_data_pd(df, partitions):
    discretized_df = df.copy()
    
    for column, breakpoints in partitions.items():
        if column in df.columns:
            # Obtener los valores mínimo y máximo para al columna 
            min_val = df[column].min()
            max_val = df[column].max() + 0.01  # sumo el máximo para que clasifique bien los extremos
            
            # Asegurarse de que los puntos de partición incluyen estos valores ajustados
            effective_breakpoints = sorted(set(breakpoints + [min_val, max_val]))
            
            # Usar pd.cut para asignar categorías a los valores basados en los puntos de partición
            discretized_df[column] = pd.cut(df[column], bins=effective_breakpoints, labels=False, right=False, include_lowest=True)
            discretized_df[column] += 1  # Ajustar índices para evitar etiquetas 0

    return discretized_df

In [5]:
class NaiveBayes:
    def fit(self, df, label_column):
        self.classes = df[label_column].unique()
        self.parameters = {}
        grouped_data = df.groupby(label_column)
        
        for cls in self.classes:
            X_cls = grouped_data.get_group(cls).drop(columns=[label_column]) #solo datos sin etiqueta
            #diccionario para guardar la prob apriori y la verosimilitud
            self.parameters[cls] = {
                'prior': len(X_cls) / len(df),
                'likelihoods': {col: self.calculate_likelihood(X_cls[col]) for col in X_cls}
            }

    def calculate_likelihood(self, feature):
        value_counts = feature.value_counts(normalize=True)
        return value_counts.to_dict()

    def predict(self, df):
        y_pred = []
        for idx, row in df.iterrows():
            posteriors = []
            for cls in self.classes:
                prior = self.parameters[cls]['prior']
                likelihood = np.prod([self.parameters[cls]['likelihoods'][col].get(val, 1e-6) for col, val in row.items()])
                posteriors.append(prior * likelihood)
            y_pred.append(self.classes[np.argmax(posteriors)])
        return pd.Series(y_pred)

In [6]:
def k_fold_cross_validation(data, k=3, shuffle=True):
    if shuffle:
        data = data.sample(frac=1).reset_index(drop=True)  # Mezclar los datos aleatoriamente
    fold_size = len(data) // k
    acc_scores = []
    target = list(data.columns)[-1]  # La etiqueta siempre es la última columna
    
    for i in range(k):
        start = i * fold_size
        end = start + fold_size if i != k - 1 else len(data)
        test_df = data.iloc[start:end]
        train_df = pd.concat([data.iloc[:start], data.iloc[end:]])
        
        nb = NaiveBayes()
        nb.fit(train_df, target)
        predictions = nb.predict(test_df.drop(columns=[target]))
        
        # Calcula la precisión como la media de las predicciones correctas
        # es un pd.series con true y false, el promedio es el accuracy
        accuracy = np.mean(predictions.values == test_df[target].values)
        
        acc_scores.append(accuracy)
        print(f"Fold {i + 1}: Accuracy = {accuracy}")
        print(f"Distribución etiquetas de entrenamiento:\n{train_df[target].value_counts()}")
        print(f"Distribución etiquetas de prueba:\n{test_df[target].value_counts()}\n")
    
    return acc_scores, np.mean(acc_scores)

In [7]:
def stratified_k_fold_cross_validation(data, k=3, shuffle=True):
    if shuffle:
        data = data.sample(frac=1, random_state=42).reset_index(drop=True)
    
    # Preparar la estructura para guardar los folds de cada clase
    folds = {i: pd.DataFrame() for i in range(k)}
    target = data.columns[-1]  # Asume que la última columna es la etiqueta
    class_proportions = data[target].value_counts(normalize=True)
    
    # Dividir los datos por clase y estratificar en folds
    for cls, proportion in class_proportions.items():
        class_data = data[data[target] == cls]
        fold_size = len(class_data) // k
        remain = len(class_data) % k
        start = 0
        
        for i in range(k):
            if i < remain:
                end = start + fold_size + 1
            else:
                end = start + fold_size
            
            folds[i] = pd.concat([folds[i], class_data.iloc[start:end]])
            start = end
    
    # Shuffle folds if required
    if shuffle:
        for i in range(k):
            folds[i] = folds[i].sample(frac=1, random_state=42).reset_index(drop=True)
    
    acc_scores = []
    
    # Evaluación cruzada en los folds
    for i in range(k):
        test_df = folds[i]
        train_df = pd.concat([folds[j] for j in range(k) if j != i]).reset_index(drop=True)
        
        nb = NaiveBayes()
        nb.fit(train_df, target)
        predictions = nb.predict(test_df.drop(columns=[target]))
        accuracy = np.mean(predictions.values == test_df[target].values)
        
        acc_scores.append(accuracy)
        print(f"Fold {i + 1}: Accuracy = {accuracy}")
    
    return acc_scores, np.mean(acc_scores)

In [8]:
csv_file = "iris" #cambiar el nombre a alguna otra base de datos
data = pd.read_csv(csv_file + '.csv')

In [9]:
data

,sepal_len,sepal_wid,petal_len,petal_wid,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [10]:
intervals = caim(data)
for key in intervals:
    print(f"Columna \"{key}\" tiene partición {intervals[key]}\n")

La columna sepal_len tiene global caim de 31.912646675358538

La columna sepal_wid tiene global caim de 23.790685128573998

La columna petal_len tiene global caim de 45.55892255892255

La columna petal_wid tiene global caim de 46.16156736446592

Columna "sepal_len" tiene partición [4.3, 5.6, 7.9]

Columna "sepal_wid" tiene partición [2.0, 3.1, 4.4]

Columna "petal_len" tiene partición [1.0, 3.0, 4.8, 6.9]

Columna "petal_wid" tiene partición [0.1, 1.0, 1.8, 2.5]



In [11]:
data_disc = discretize_data_pd(data,intervals)

In [12]:
data_disc

,sepal_len,sepal_wid,petal_len,petal_wid,class
0,1,2,1,1,Iris-setosa
1,1,1,1,1,Iris-setosa
2,1,2,1,1,Iris-setosa
3,1,2,1,1,Iris-setosa
4,1,2,1,1,Iris-setosa
...,...,...,...,...,...
145,2,1,3,3,Iris-virginica
146,2,1,3,3,Iris-virginica
147,2,1,3,3,Iris-virginica
148,2,2,3,3,Iris-virginica


In [13]:
data_disc.to_csv(csv_file +'_disc.csv', index=False)  

In [14]:
nb = NaiveBayes()
target = list(data_disc.columns)[-1]
nb.fit(data, target)
predictions = nb.predict(data.drop(columns=[target]))

In [15]:
scores, mean_accuracy = k_fold_cross_validation(data_disc, k=3)
print("Precisión en cada fold:", scores)
print("Precisión media:", mean_accuracy)

Fold 1: Accuracy = 0.94
Distribución etiquetas de entrenamiento:
Iris-virginica     34
Iris-versicolor    33
Iris-setosa        33
Name: class, dtype: int64
Distribución etiquetas de prueba:
Iris-setosa        17
Iris-versicolor    17
Iris-virginica     16
Name: class, dtype: int64

Fold 2: Accuracy = 0.96
Distribución etiquetas de entrenamiento:
Iris-versicolor    35
Iris-setosa        34
Iris-virginica     31
Name: class, dtype: int64
Distribución etiquetas de prueba:
Iris-virginica     19
Iris-setosa        16
Iris-versicolor    15
Name: class, dtype: int64

Fold 3: Accuracy = 0.88
Distribución etiquetas de entrenamiento:
Iris-virginica     35
Iris-setosa        33
Iris-versicolor    32
Name: class, dtype: int64
Distribución etiquetas de prueba:
Iris-versicolor    18
Iris-setosa        17
Iris-virginica     15
Name: class, dtype: int64

Precisión en cada fold: [0.94, 0.96, 0.88]
Precisión media: 0.9266666666666666


In [16]:
s_scores = stratified_k_fold_cross_validation(data_disc, k=3, shuffle=True)
print(f"Presición media: {s_scores[1]}")

Fold 1: Accuracy = 0.9803921568627451
Fold 2: Accuracy = 0.9215686274509803
Fold 3: Accuracy = 0.9375
Presición media: 0.9464869281045751
